In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from bs4 import BeautifulSoup
import re
import nltk

In [2]:
train = pd.read_csv('sorting_hat2.csv')
train

,id,name,birth,gender,hair,eye,House
0,3,Angelina Johnson,10,2,black,brown,Gryffindor
1,4,Anthony Goldstein,9,1,blond,brown,Ravenclaw
2,5,Blaise Zabini,9,1,black,brown,Slytherin
3,7,Cedric Diggory,9,1,brown,grey,Hufflepuff
4,9,Colin Creevey,3,1,brown,brown,Gryffindor
5,14,Draco Malfoy,6,1,blond,grey,Slytherin
6,17,Ernie Macmillan,4,1,blond,brown,Hufflepuff
7,19,Fred Weasley,4,1,red,brown,Gryffindor
8,35,Luna Lovegood,6,2,blond,grey,Ravenclaw
9,44,Neville Longbottom,7,1,blond,brown,Gryffindor


In [3]:
feature = pd.DataFrame(np.zeros(shape=(len(train),26+1+1+26+26)))
feature

,0,1,2,3,4,5,6,7,8,9,...,70,71,72,73,74,75,76,77,78,79
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
ab = 'abcdefghijklmnopqrstuvwxyz'
for i in feature.index:
    feature.iloc[i,26] = train.loc[i,'birth']
    feature.iloc[i,27] = train.loc[i,'gender']
    for char in ab:
        feature.iloc[i,ab.index(char)] = train.loc[i,'name'].lower().count(char)
        feature.iloc[i,28+ab.index(char)] = train.loc[i,'hair'].lower().count(char)
        feature.iloc[i,54+ab.index(char)] = train.loc[i,'eye'].lower().count(char)
feature

,0,1,2,3,4,5,6,7,8,9,...,70,71,72,73,74,75,76,77,78,79
0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,2.0,2.0,1.0,0.0,2.0,0.0,2.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,2.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6,2.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7,1.0,0.0,0.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9,0.0,1.0,0.0,0.0,2.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [5]:
y = [1,3,4,2,1,4,2,1,3,1,3,1,3]

In [6]:
model = RandomForestClassifier(n_estimators=500)
model.fit(feature.values, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [7]:
y_test = model.predict(feature.values)

In [8]:
sum(y_test!=y)

0

In [9]:
name = 'Robert Carl Minnich'
birth = 11
gender = 1
hair = 'blond'
eye = 'brown'

In [10]:
f = np.zeros(26+1+1+26+26)
f[26] = birth
f[27] = gender
for char in ab:
    f[ab.index(char)] = name.lower().count(char)
    f[28+ab.index(char)] = hair.lower().count(char)
    f[54+ab.index(char)] = eye.lower().count(char)
f

array([  1.,   1.,   2.,   0.,   1.,   0.,   0.,   1.,   2.,   0.,   0.,
         1.,   1.,   2.,   1.,   0.,   0.,   3.,   0.,   1.,   0.,   0.,
         0.,   0.,   0.,   0.,  11.,   1.,   0.,   1.,   0.,   1.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   1.,   1.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   1.,   1.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,   1.,
         0.,   0.,   0.])

In [11]:
model.predict(f)

/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([2])